# Testing Selenium Scraping on Immoweb

## First do the usefulls imports

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import json
import time

## Manage the cookies on Immoweb

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.immoweb.be/en")
time.sleep(3)
cookies = driver.find_element(By.XPATH,'/html/body/div[3]/div[4]/div[2]/div/div[2]/div[1]/button')
cookies.click()
driver.close()

#### Can be improve ...



## Create urls data saving in a .json file

In [ ]:
def save_cache (provinces_urls_dict) :
    
    with open("cache_urls.json","w") as my_file :
        json.dump(provinces_urls_dict,my_file)

## Extract URLs of Houses and Appartments for sale inside an Immoweb page

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.immoweb.be/en")
time.sleep(3)
cookies = driver.find_element(By.XPATH,'/html/body/div[3]/div[4]/div[2]/div/div[2]/div[1]/button')
cookies.click()

H_A_page_urls_set = set()
driver.get("https://www.immoweb.be/en/search/house-and-apartment/for-sale/brussels/province?countries=BE&isALifeAnnuitySale=false&page=1&orderBy=relevance")
H_A_driver_list = driver.find_elements(By.XPATH,'/html/body/div[1]/div[1]/div/main/div/div[2]/div/div[3]/div/div[1]/div[1]/div[1]/ul/li[*]/article/div[1]/h2/a')
H_A_page_urls_set = {H_A.get_attribute("href") for H_A in H_A_driver_list}
print(H_A_page_urls_set)
driver.close()

### Create the appropriate function to extract URLs

In [ ]:
def get_houses_appartments_page_urls(driver) :
    
    H_A_driver_list = driver.find_elements(By.XPATH,'/html/body/div[1]/div[1]/div/main/div/div[2]/div/div[3]/div/div[1]/div[1]/div[1]/ul/li[*]/article/div[1]/h2/a')
    H_A_page_urls_set = {H_A.get_attribute("href") for H_A in H_A_driver_list}
    
    return H_A_page_urls_set

## Loop over all the Immoweb pages (Houses and Appartements for sales) for one Province

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(f"https://www.immoweb.be/en")
time.sleep(3)
cookies = driver.find_element(By.XPATH,'/html/body/div[3]/div[4]/div[2]/div/div[2]/div[1]/button')
cookies.click()

H_A_page_urls_set = set()
driver.get(f"https://www.immoweb.be/en/search/house-and-apartment/for-sale/brussels/province?countries=BE&isALifeAnnuitySale=false&page=1&orderBy=relevance")
max_page = int(driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div/main/div/div[2]/div/div[3]/div/div[1]/div[1]/div[1]/div/div[1]/div/nav/ul/li[4]/a/span[2]').text)
#extract urls

for page in range(2,max_page+1) :

    driver.get(f"https://www.immoweb.be/en/search/house-and-apartment/for-sale/brussels/province?countries=BE&isALifeAnnuitySale=false&page={page}&orderBy=relevance")
    #extract urls

driver.close()

### Create the appropriate function to loop over all pages for one Province and extract URLs

In [ ]:
def get_houses_appartments_pages_urls(driver,province) :
    
    H_A_pages_urls_set = set()
    driver.get(f"https://www.immoweb.be/en/search/house-and-apartment/for-sale/{province}/province?countries=BE&isALifeAnnuitySale=false&page=1&orderBy=relevance")
    max_page = int(driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div/main/div/div[2]/div/div[3]/div/div[1]/div[1]/div[1]/div/div[1]/div/nav/ul/li[4]/a/span[2]').text)

    H_A_pages_urls_set.update(get_houses_appartments_page_urls(driver))

    for page in range(2,max_page+1) :

        driver.get(f"https://www.immoweb.be/en/search/house-and-apartment/for-sale/{province}/province?countries=BE&isALifeAnnuitySale=false&page={page}&orderBy=relevance")

        H_A_pages_urls_set.update(get_houses_appartments_page_urls(driver))

    return H_A_pages_urls_set

## Loop over all the Immoweb Provinces pages (Houses and Appartements for sales)

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(f"https://www.immoweb.be/en")
time.sleep(3)
cookies = driver.find_element(By.XPATH,'/html/body/div[3]/div[4]/div[2]/div/div[2]/div[1]/button')
cookies.click()

belgium_provinces = ("brussels","liege","namur","luxembourg","hainaut","walloon-brabant","antwerp","limburg","west-flanders","east-flanders","flemish-brabant")
H_A_pages_urls_set = set()

for province in belgium_provinces :
    driver.get(f"https://www.immoweb.be/en/search/house-and-apartment/for-sale/{province}/province?countries=BE&isALifeAnnuitySale=false&page=1&orderBy=relevance")
    # loop on pages and extract urls

driver.close()

### Create the appropriate function to loop over all Provinces pages

In [ ]:
def get_houses_appartments_site_urls() :

    provinces_urls_dict = {}
    belgium_provinces = ["brussels","liege","namur","luxembourg","hainaut","walloon-brabant","antwerp","limburg","west-flanders","east-flanders","flemish-brabant"]

    for province in belgium_provinces :
        provinces_urls_dict[province] = list(get_houses_appartments_pages_urls(driver,province))

    return  provinces_urls_dict

## Extract House or Appartement Data from an Immoweb property description page

## Create the appropriate Script

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import json



def save_cache (for_sell_immoweb_urls) :
    
    with open("cache_urls.json","w") as my_file :
        json.dump(for_sell_immoweb_urls,my_file)


def get_houses_appartments_page_urls(driver) :
    
    H_A_driver_list = driver.find_elements(By.XPATH,'/html/body/div[1]/div[1]/div/main/div/div[2]/div/div[3]/div/div[1]/div[1]/div[1]/ul/li[*]/article/div[1]/h2/a')
    H_A_page_urls_set = {H_A.get_attribute("href") for H_A in H_A_driver_list}
    
    return H_A_page_urls_set


def get_houses_appartments_pages_urls(driver,province) :
    
    H_A_pages_urls_set = set()
    driver.get(f"https://www.immoweb.be/en/search/house-and-apartment/for-sale/{province}/province?countries=BE&isALifeAnnuitySale=false&page=1&orderBy=relevance")
    max_page = int(driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div/main/div/div[2]/div/div[3]/div/div[1]/div[1]/div[1]/div/div[1]/div/nav/ul/li[4]/a/span[2]').text)

    H_A_pages_urls_set.update(get_houses_appartments_page_urls(driver))

    for page in range(2,max_page+1) :

        driver.get(f"https://www.immoweb.be/en/search/house-and-apartment/for-sale/{province}/province?countries=BE&isALifeAnnuitySale=false&page={page}&orderBy=relevance")

        H_A_pages_urls_set.update(get_houses_appartments_page_urls(driver))

    return H_A_pages_urls_set


def get_houses_appartments_site_urls() :

    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get("https://www.immoweb.be/en")
    time.sleep(3)
    cookies = driver.find_element(By.XPATH,'/html/body/div[3]/div[4]/div[2]/div/div[2]/div[1]/button')
    cookies.click()

    provinces_urls_dict = {}
    belgium_provinces = ["brussels","liege","namur","luxembourg","hainaut","walloon-brabant","antwerp","limburg","west-flanders","east-flanders","flemish-brabant"]

    for province in belgium_provinces :
        provinces_urls_dict[province] = list(get_houses_appartments_pages_urls(driver,province))

    driver.close()

    return  provinces_urls_dict

   

if __name__ == '__main__' :

    for_sell_immoweb_urls = get_houses_appartments_site_urls()
    save_cache(for_sell_immoweb_urls)

### Improve the Script with Concurrency and Parallelism

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from multiprocessing import Pool
import json



def save_cache (H_A_sell_immoweb_urls) :
    
    with open("cache_urls.json","w") as my_file :
        json.dump(H_A_sell_immoweb_urls,my_file)


def get_houses_appartments_page_urls(driver) :
    
    H_A_driver_list = driver.find_elements(By.XPATH,'/html/body/div[1]/div[1]/div/main/div/div[2]/div/div[3]/div/div[1]/div[1]/div[1]/ul/li[*]/article/div[1]/h2/a')
    H_A_page_urls_set = {H_A.get_attribute("href") for H_A in H_A_driver_list}
    
    return H_A_page_urls_set


def get_houses_appartments_pages_urls(province) :
    
    H_A_pages_urls_set = set()
    driver = webdriver.Chrome()
    page = 1
    
    while True :

        driver.get(f"https://www.immoweb.be/en/search/house-and-apartment/for-sale/{province}/province?countries=BE&isALifeAnnuitySale=false&page={page}&orderBy=relevance")

        try :
            
            if driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div/main/div/div[2]/div/div[2]/div/div/div[2]/h1').text == "Sorry, no matching results." :
                    break
        except :
        
            try :
                if driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/main/div/div/div[2]/p[1]/strong').text == "500 server error" :
                    break
            except :

                H_A_pages_urls_set.update(get_houses_appartments_page_urls(driver))
            
                page += 1
    
    driver.close()

    return H_A_pages_urls_set


def get_houses_appartments_site_urls() :
    
    belgium_provinces = ["brussels","liege","namur","luxembourg","hainaut","walloon-brabant","antwerp","limburg","west-flanders","east-flanders","flemish-brabant"]

    with Pool(2) as pool:

        site_urls_pool_output = list(pool.map(get_houses_appartments_pages_urls,belgium_provinces))

    provinces_urls_dict = {province : list(site_urls_pool_output[index]) for index,province in enumerate(belgium_provinces)}
    
    return  provinces_urls_dict



if __name__ == '__main__' :

    H_A_sell_immoweb_urls = get_houses_appartments_site_urls()
    save_cache(H_A_sell_immoweb_urls)

## Create the appropriate Class & Import

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

class Immoweb_H_A_Data_pool() :
    pass

#to do